In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from typing import Tuple, Dict, Any
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import lightgbm as lgb
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
import time


raw_df = pd.read_csv("drive/MyDrive/Colab Notebooks/train.csv")


raw_df = raw_df.drop(columns=["id", "CustomerId", "Surname"], errors="ignore")


def split_train_val(df: pd.DataFrame, target_col: str, test_size: float = 0.2, random_state: int = 42) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Split the dataframe into training and validation sets.

    Args:
        df (pd.DataFrame): The raw dataframe.
        target_col (str): The target column for stratification.
        test_size (float): The proportion of the dataset to include in the validation split.
        random_state (int): Random state for reproducibility.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: Training and validation dataframes.
    """
    train_df, val_df = train_test_split(df, test_size=test_size, random_state=random_state, stratify=df[target_col])
    return train_df, val_df


def separate_inputs_targets(df: pd.DataFrame, input_cols: list, target_col: str) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Separate inputs and targets from the dataframe.

    Args:
        df (pd.DataFrame): The dataframe.
        input_cols (list): List of input columns.
        target_col (str): Target column.

    Returns:
        Tuple[pd.DataFrame, pd.Series]: DataFrame of inputs and Series of targets.
    """
    inputs = df[input_cols].copy()
    targets = df[target_col].copy()
    return inputs, targets

In [4]:
input_cols = [c for c in raw_df.columns if c != "Exited"]

train_df, val_df = split_train_val(raw_df, target_col="Exited")

train_inputs, train_targets = separate_inputs_targets(train_df, input_cols, target_col="Exited")
val_inputs, val_targets = separate_inputs_targets(val_df, input_cols, target_col="Exited")

In [5]:
for col in train_inputs.select_dtypes(include="object").columns:
    train_inputs[col] = train_inputs[col].astype("category")
    val_inputs[col] = val_inputs[col].astype("category")

print("Train shape:", train_inputs.shape, "Validation shape:", val_inputs.shape)
print("Categorical:", train_inputs.select_dtypes(include="category").columns.tolist())

Train shape: (12000, 10) Validation shape: (3000, 10)
Категоріальні ознаки: ['Geography', 'Gender']


In [7]:
xgb_clf = XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method="hist",  # пришвидшує
    use_label_encoder=False,
    eval_metric="auc",
    enable_categorical=True
)
xgb_clf.fit(train_inputs, train_targets)

train_preds = xgb_clf.predict_proba(train_inputs)[:,1]
val_preds = xgb_clf.predict_proba(val_inputs)[:,1]

print("Train AUROC:", roc_auc_score(train_targets, train_preds))
print("Validation AUROC:", roc_auc_score(val_targets, val_preds))


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [12:49:27] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Train AUROC: 0.9613667982312049
Validation AUROC: 0.9338102750531586


**The baseline XGBoost model showed high performance, exceeding the results of Decision Trees.
The difference between the training and validation sets is small, which means the model generalizes well and does not suffer from significant overfitting. It has both low bias and low variance, and even with default settings, it outperforms the Decision Tree.**

In [9]:
space = {
    'max_depth': hp.choice('max_depth', np.arange(3, 10, dtype=int)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'n_estimators': hp.choice('n_estimators', np.arange(100, 400, 50, dtype=int)),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'gamma': hp.uniform('gamma', 0, 5),
    'min_child_weight': hp.choice('min_child_weight', [1, 3, 5, 7])
}

# objective function

def objective(params):
    clf = XGBClassifier(
        **params,
        tree_method="hist",
        enable_categorical=True,
        missing=np.nan,
        eval_metric="auc",
        use_label_encoder=False,
        random_state=42
    )
    clf.fit(train_inputs, train_targets)
    val_preds = clf.predict_proba(val_inputs)[:, 1]
    auc = roc_auc_score(val_targets, val_preds)
    return {'loss': -auc, 'status': STATUS_OK}

# hyperopt

trials = Trials()
best_params = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=20,
    trials=trials,
    rstate=np.random.default_rng(42)
)

print("Best parameters:", best_params)

# restoring the actual values of hp.choice

n_estimators_choices = list(np.arange(100, 400, 50, dtype=int))
max_depth_choices = list(np.arange(3, 10, dtype=int))
min_child_weight_choices = [1, 3, 5, 7]

final_clf = XGBClassifier(
    max_depth=max_depth_choices[best_params['max_depth']],
    learning_rate=best_params['learning_rate'],
    n_estimators=n_estimators_choices[best_params['n_estimators']],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    gamma=best_params['gamma'],
    min_child_weight=min_child_weight_choices[best_params['min_child_weight']],
    tree_method="hist",
    enable_categorical=True,
    missing=np.nan,
    eval_metric="auc",
    use_label_encoder=False,
    random_state=42
)

# final model

final_clf.fit(train_inputs, train_targets)

train_preds = final_clf.predict_proba(train_inputs)[:, 1]
val_preds = final_clf.predict_proba(val_inputs)[:, 1]

print("Train AUROC:", roc_auc_score(train_targets, train_preds))
print("Validation AUROC:", roc_auc_score(val_targets, val_preds))

  5%|▌         | 1/20 [00:00<00:02,  6.98trial/s, best loss: -0.936366691816997]

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [13:16:55] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [13:16:56] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



 15%|█▌        | 3/20 [00:00<00:03,  4.69trial/s, best loss: -0.9372144865903012]

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [13:16:56] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [13:16:56] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



 20%|██        | 4/20 [00:00<00:03,  4.79trial/s, best loss: -0.9372144865903012]

...

 95%|█████████▌| 19/20 [00:03<00:00,  4.83trial/s, best loss: -0.9372144865903012]

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [13:16:59] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)

/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [13:16:59] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



100%|██████████| 20/20 [00:03<00:00,  5.25trial/s, best loss: -0.9372144865903012]
Найкращі гіперпараметри: {'colsample_bytree': np.float64(0.9293728051431607), 'gamma': np.float64(2.86364059503943), 'learning_rate': np.float64(0.05558211316295016), 'max_depth': np.int64(1), 'min_child_weight': np.int64(0), 'n_estimators': np.int64(1), 'subsample': np.float64(0.658785114914011)}


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [13:16:59] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Train AUROC: 0.9464862054315915
Validation AUROC: 0.9372144865903012


**Optimization of XGBoost with Hyperopt improved the model’s performance. The difference between Train and Validation is small, indicating a good balance of bias and variance and no significant overfitting.**

In [18]:
categorical_features = [
    col for col in train_inputs.columns if train_inputs[col].dtype.name == "category"
]
cat_feature_indexes = [
    train_inputs.columns.get_loc(col) for col in categorical_features
]

# LightGBM

lgb_clf = lgb.LGBMClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=-1,             # without depth limitation
    subsample=0.8,            # fraction of data per tree
    colsample_bytree=0.8,     # fraction of features per tree
    random_state=42
)

lgb_clf.fit(
    train_inputs,
    train_targets,
    eval_set=[(val_inputs, val_targets)],
    eval_metric="auc",
    categorical_feature=cat_feature_indexes
)



train_preds = lgb_clf.predict_proba(train_inputs)[:, 1]
val_preds = lgb_clf.predict_proba(val_inputs)[:, 1]

train_auc = roc_auc_score(train_targets, train_preds)
val_auc = roc_auc_score(val_targets, val_preds)

print("Train AUROC:", train_auc)
print("Validation AUROC:", val_auc)


[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001014 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 843
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561
Train AUROC: 0.9905229017752557
Validation AUROC: 0.9292523492694972


**The baseline LightGBM model demonstrated fairly high performance, but it lags behind the optimized XGBoost. The training AUROC is noticeably higher than the validation AUROC, which indicates some degree of overfitting and high variance.**

In [19]:
space_lgb = {
    'num_leaves': hp.choice('num_leaves', np.arange(20, 150, dtype=int)),
    'max_depth': hp.choice('max_depth', np.arange(3, 12, dtype=int)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'n_estimators': hp.choice('n_estimators', np.arange(100, 400, 50, dtype=int)),
    'subsample': hp.uniform('subsample', 0.7, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.7, 1.0),
    'min_child_samples': hp.choice('min_child_samples', [5, 10, 20, 30])
}

def objective_lgb(params):
    clf = lgb.LGBMClassifier(
        **params,
        random_state=42
    )
    clf.fit(
        train_inputs,
        train_targets,
        eval_set=[(val_inputs, val_targets)],
        eval_metric="auc",
        categorical_feature=cat_feature_indexes,
        callbacks=[lgb.log_evaluation(0)]
    )
    val_preds = clf.predict_proba(val_inputs)[:, 1]
    auc = roc_auc_score(val_targets, val_preds)
    return {'loss': -auc, 'status': STATUS_OK}

# hyperopt

trials_lgb = Trials()
best_lgb = fmin(
    fn=objective_lgb,
    space=space_lgb,
    algo=tpe.suggest,
    max_evals=10,
    trials=trials_lgb,
    rstate=np.random.default_rng(42)
)

print("Best parameters LightGBM:", best_lgb)

# restoring the actual values of hp.choice

num_leaves_choices = list(np.arange(20, 150, dtype=int))
max_depth_choices = list(np.arange(3, 12, dtype=int))
n_estimators_choices = list(np.arange(100, 400, 50, dtype=int))
min_child_samples_choices = [5, 10, 20, 30]

final_lgb_clf = lgb.LGBMClassifier(
    num_leaves=num_leaves_choices[best_lgb['num_leaves']],
    max_depth=max_depth_choices[best_lgb['max_depth']],
    learning_rate=best_lgb['learning_rate'],
    n_estimators=n_estimators_choices[best_lgb['n_estimators']],
    subsample=best_lgb['subsample'],
    colsample_bytree=best_lgb['colsample_bytree'],
    min_child_samples=min_child_samples_choices[best_lgb['min_child_samples']],
    random_state=42
)

# final model

final_lgb_clf.fit(
    train_inputs,
    train_targets,
    eval_set=[(val_inputs, val_targets)],
    eval_metric="auc",
    categorical_feature=cat_feature_indexes,
    callbacks=[lgb.log_evaluation(0)]
)

train_preds = final_lgb_clf.predict_proba(train_inputs)[:, 1]
val_preds = final_lgb_clf.predict_proba(val_inputs)[:, 1]

train_auc = roc_auc_score(train_targets, train_preds)
val_auc = roc_auc_score(val_targets, val_preds)

print("Train AUROC:", train_auc)
print("Validation AUROC:", val_auc)


Категоріальні ознаки: ['Geography', 'Gender']
Індекси категоріальних ознак: [1, 2]
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001160 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 843
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.203500 -> initscore=-1.364561
[LightGBM] [Info] Start training from score -1.364561
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Number of positive: 2442, number of negative: 9558
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhe

**The LightGBM model delivered a good result, better than in the previous task. Despite numerous warnings about the absence of further splits, the selected hyperparameters turned out to be quite effective.**



**XGBoost optimized with Hyperopt was chosen, as it showed the best AUROC performance in the experiments.**

In [22]:
test_df = pd.read_csv("drive/MyDrive/Colab Notebooks/test.csv")
for col in test_df.select_dtypes(include="object").columns:
    test_df[col] = test_df[col].astype("category")

test_preds = final_clf.predict_proba(test_df[input_cols])[:,1]

sample_submission = pd.read_csv("drive/MyDrive/Colab Notebooks/sample_submission.csv")
sample_submission["Exited"] = test_preds
sample_submission.to_csv("submission_boosting.csv", index=False)

In [23]:
from google.colab import files
files.download("submission_boosting.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>